# Google Drive 연결 & 해당 경로 이동

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
cd /content/drive/MyDrive/YOLOv3

# 구글 드라이브에 darknet clone
- Alexey/darknet github 를 사용
- 구글 드라이브에 darknet 파일이 없는 경우에만 실행 (처음에만 실행)

In [ ]:
!git clone https://github.com/AlexeyAB/darknet.git

# 마스크 인식 YOLOv3-tiny 모델 생성
- **YOLOv3-tiny Custom 모델 생성 과정**
> 1. 마스크 착용자 & 미착용자 데이터 셋 수집 (각 class 당 최소 5000장)
> 2. 이미지 labeling (openLabeling 오픈소스 Tool 사용)
> 3. yolov3-tiny_obj.cfg 파일 수정 (class 개수에 맞게 수정  
> 4. obj.data & obj.names 파일 생성
> 5. process.py 파일 생성  
> 6. 생성한 모든 파일 구글 드라이브 yolo 폴더에 업로드 (obj.zip, obj.data, obj.names, process.py) 


- [참고 링크](https://ichi.pro/ko/sayongja-jijeong-yolov4-gaeche-tamjigi-gyoyug-google-colab-sayong-6710443722856)

1. Makefile에서 OPENCV 및 GPU 활성화

In [ ]:
%cd darknet/

In [ ]:
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!sed -i 's/CUDNN_HALF=0/CUDNN_HALF=1/' Makefile
!sed -i 's/LIBSO=0/LIBSO=1/' Makefile

In [ ]:
!make

2. 관계없는 데이터 및 cfg 폴더 정리

In [ ]:
%cd data/
!find -maxdepth 1 -type f -exec rm -rf {} \;
%cd ..
%rm -rf cfg/
%mkdir cfg

3. 업로드한 obj.zip 파일 압축 해제 (1번만 실행해도 된다.) 

In [ ]:
!unzip /content/drive/MyDrive/YOLOv3/obj.zip -d data/

4. 작성한 cfg파일과 obj.names, obj.data 파일을 각각의 위치로 복사 
- cfg 파일 위치: darknet/cfg
- obj.names & obj.data 파일 위치: darknet/data

In [ ]:
# darknet/cfg
!cp /content/drive/MyDrive/YOLOv3/yolov3-tiny_obj.cfg cfg

# darknet/data
!cp /content/drive/MyDrive/YOLOv3/obj.names data
!cp /content/drive/MyDrive/YOLOv3/obj.data  data

5. 작성한 process.py 파일을 darknet 폴더로 복사

In [ ]:
!cp /content/drive/MyDrive/YOLOv3/process.py .

6. process.py 실행

In [ ]:
!python process.py

In [ ]:
# darknet/data 폴더 안의 파일 확인 (제대로 파일이 들어 있는지 확인)
!ls data/

7. 학습에 필요한 **yolov3-tiny.conv.11** 파일 다운로드
- darknet 폴더 안에 업로드
- [다운로드 링크](https://drive.google.com/file/d/18v36esoXCh-PsOKwyP2GWrpYDptDY8Zf/view)  (출처: Alexey/darknet github)


# 모델 학습

## Colab 연결 해지를 막는 자바 스크립트:  
1. (Ctrl + Shift + i) 를 눌러 콘솔로 이동 - Chrome 기준
2. 다음 셀의 코드를 붙여넣고 Enter

In [ ]:
function ClickConnect(){
console.log("Working"); 
document
  .querySelector('#top-toolbar > colab-connect-button')
  .shadowRoot.querySelector('#connect')
  .click() 
}
setInterval(ClickConnect,60000)

## 학습

- 처음 학습 시작할 때:

In [ ]:
!./darknet detector train data/obj.data cfg/yolov3-tiny_obj.cfg yolov3-tiny.conv.11 -dont_show -map

# mAP 출력 버전 (Colab에서는 안 돌아간다..)
# !./darknet detector train data/obj.data cfg/yolov3-tiny_obj.cfg yolov3-tiny.conv.11 -map

- 중간에 런타임이 끊겼을 때:

In [ ]:
!./darknet detector train data/obj.data cfg/yolov3-tiny_obj.cfg /content/drive/MyDrive/YOLOv3/training/yolov3-tiny_obj_last.weights -dont_show -map

# 모델 학습 결과 확인

# chart.png
- 학습되는 모든 과정에서의 mAP 와 loss의 변화를 그래프로 확인할 수 있다 .

In [ ]:
import cv2
import matplotlib.pyplot as plt

def imShow(path):
  %matplotlib inline
  image = cv2.imread(path)
  height, width = image.shape[:2]
  resized_image = cv2.resize(image,(3*width, 3*height), interpolation = cv2.INTER_CUBIC)
  fig = plt.gcf()
  fig.set_size_inches(18, 10)
  plt.axis("off")
  plt.imshow(cv2.cvtColor(resized_image, cv2.COLOR_BGR2RGB))
  plt.show()

In [ ]:
imShow('chart.png')

## mAP (평균 정밀도) 확인
- training 폴더에 1000번 반복될 때마다 저장된 가중치들에 대해 mAP를 확인할 수 있다. (예: yolov3-custom_4000.weights)
- yolov3-custom_best.weights : 최상의 결과를 내는 가중치
- mAP는 높을수록 좋다.

In [ ]:
!./darknet detector map data/obj.data cfg/yolov3-tiny_obj.cfg /content/drive/MyDrive/YOLOv3/training/yolov3-tiny_obj_best.weights -points 0

## cfg 파일 Test 주석 처리 풀고 테스트하기

In [ ]:
%cd cfg
!sed -i 's/batch=64/batch=1/' yolov3-tiny_obj.cfg
!sed -i 's/subdivisions=16/subdivisions=1/' yolov3-tiny_obj.cfg
%cd ..